**Import packages and stopwords**

In [1]:
!pip install pattern

You are using pip version 19.2.2, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install -U gensim

Requirement already up-to-date: gensim in /home/nbuser/anaconda2_501/lib/python2.7/site-packages (3.8.0)
You are using pip version 19.2.2, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection u'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-t

True

In [4]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection u'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/nbuser/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-t

True

In [5]:
from gensim.models import LdaModel, LdaMulticore
import gensim.downloader as api
from gensim.utils import simple_preprocess, lemmatize
from nltk.corpus import stopwords
import re
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
stop_words = stopwords.words('english')
stop_words = stop_words + ['com', 'edu', 'subject', 'lines', 'organization', 'would', 'article', 'could']

**Import the dataset and get the text and real topic of each news article**

In [6]:
dataset = api.load("text8")
data = [d for d in dataset]

**Prepare Data (Remove stopwords and lemmatize)**

In [7]:
data_processed = []

for i, doc in enumerate(data[:100]):
    doc_out = []
    for wd in doc:
        if wd not in stop_words:  # remove stopwords
            lemmatized_word = lemmatize(wd, allowed_tags=re.compile('(NN|JJ|RB)'))  # lemmatize
            if lemmatized_word:
                doc_out = doc_out + [lemmatized_word[0].split(b'/')[0].decode('utf-8')]
        else:
            continue
    data_processed.append(doc_out)

# Print a small sample    
print(data_processed[0][:5]) 

[u'anarchism', u'originated', u'term', u'abuse', u'first']


**Create the Inputs of LDA model: Dictionary and Corpus**

In [9]:
from gensim import corpora
dct = corpora.Dictionary(data_processed)
corpus = [dct.doc2bow(line) for line in data_processed]

2019-08-27 18:36:25,238 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-08-27 18:36:27,146 : INFO : built Dictionary(40074 unique tokens: [u'fawn', u'homomorphism', u'schlegel', u'nunnery', u'parallelogram']...) from 100 documents (total 425549 corpus positions)


**Train the LDA model**

In [10]:
lda_model = LdaMulticore(corpus=corpus,
                         id2word=dct,
                         random_state=100,
                         num_topics=7,
                         passes=10,
                         chunksize=1000,
                         batch=False,
                         alpha='asymmetric',
                         decay=0.5,
                         offset=64,
                         eta=None,
                         eval_every=0,
                         iterations=100,
                         gamma_threshold=0.001,
                         per_word_topics=True)

# save the model
lda_model.save('lda_model.model')

# See the topics
lda_model.print_topics(-1)
# [(0, '0.001*"also" + 0.000*"first" + 0.000*"state" + 0.000*"american" + 0.000*"time" + 0.000*"book" + 0.000*"year" + 0.000*"many" + 0.000*"person" + 0.000*"new"'),
#  (1, '0.001*"also" + 0.001*"state" + 0.001*"ammonia" + 0.000*"first" + 0.000*"many" + 0.000*"american" + 0.000*"war" + 0.000*"time" + 0.000*"year" + 0.000*"name"'),
#  (2, '0.005*"also" + 0.004*"american" + 0.004*"state" + 0.004*"first" + 0.003*"year" + 0.003*"many" + 0.003*"time" + 0.003*"new" + 0.003*"war" + 0.003*"person"'),
#  (3, '0.001*"atheism" + 0.001*"also" + 0.001*"first" + 0.001*"atheist" + 0.001*"american" + 0.000*"god" + 0.000*"state" + 0.000*"many" + 0.000*"new" + 0.000*"year"'),
#  (4, '0.001*"state" + 0.001*"also" + 0.001*"many" + 0.000*"world" + 0.000*"agave" + 0.000*"time" + 0.000*"new" + 0.000*"war" + 0.000*"god" + 0.000*"person"'),
#  (5, '0.001*"also" + 0.001*"abortion" + 0.001*"first" + 0.001*"american" + 0.000*"state" + 0.000*"many" + 0.000*"year" + 0.000*"time" + 0.000*"war" + 0.000*"person"'),
#  (6, '0.005*"also" + 0.004*"first" + 0.003*"time" + 0.003*"many" + 0.003*"state" + 0.003*"world" + 0.003*"american" + 0.003*"person" + 0.003*"apollo" + 0.003*"language"')]

2019-08-27 18:37:07,264 : INFO : using asymmetric alpha [0.26219156, 0.19027454, 0.14931786, 0.12287004, 0.104381524, 0.090729296, 0.080235206]
2019-08-27 18:37:07,286 : INFO : using symmetric eta at 0.142857142857
2019-08-27 18:37:07,313 : INFO : using serial LDA version on this node
2019-08-27 18:37:07,548 : INFO : running online LDA training, 7 topics, 10 passes over the supplied corpus of 100 documents, updating every 1000 documents, evaluating every ~0 documents, iterating 100x with a convergence threshold of 0.001000
2019-08-27 18:37:07,770 : INFO : training LDA model using 1 processes
2019-08-27 18:37:16,450 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #100/100, outstanding queue size 1
2019-08-27 18:37:22,653 : INFO : topic #6 (0.080): 0.001*"also" + 0.001*"american" + 0.001*"state" + 0.001*"person" + 0.001*"time" + 0.001*"world" + 0.001*"year" + 0.001*"war" + 0.001*"many" + 0.001*"first"
2019-08-27 18:37:22,657 : INFO : topic #5 (0.091): 0.001*"also" + 0.00

[(0,
  u'0.001*"also" + 0.000*"first" + 0.000*"state" + 0.000*"american" + 0.000*"time" + 0.000*"book" + 0.000*"year" + 0.000*"many" + 0.000*"person" + 0.000*"new"'),
 (1,
  u'0.001*"also" + 0.001*"state" + 0.001*"ammonia" + 0.000*"first" + 0.000*"many" + 0.000*"american" + 0.000*"war" + 0.000*"time" + 0.000*"year" + 0.000*"name"'),
 (2,
  u'0.005*"also" + 0.004*"american" + 0.004*"state" + 0.004*"first" + 0.003*"year" + 0.003*"many" + 0.003*"time" + 0.003*"new" + 0.003*"war" + 0.003*"person"'),
 (3,
  u'0.001*"atheism" + 0.001*"also" + 0.001*"first" + 0.000*"atheist" + 0.000*"american" + 0.000*"god" + 0.000*"state" + 0.000*"many" + 0.000*"new" + 0.000*"year"'),
 (4,
  u'0.001*"state" + 0.001*"also" + 0.001*"many" + 0.000*"world" + 0.000*"agave" + 0.000*"time" + 0.000*"new" + 0.000*"war" + 0.000*"god" + 0.000*"person"'),
 (5,
  u'0.001*"also" + 0.001*"abortion" + 0.001*"first" + 0.001*"american" + 0.000*"state" + 0.000*"many" + 0.000*"year" + 0.000*"time" + 0.000*"war" + 0.000*"person"

**Interpret the LDA Topic Model’s output**

In [11]:
# Reference: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/topic_methods.ipynb
for c in lda_model[corpus[5:8]]:
    print("Document Topics      : ", c[0])      # [(Topics, Perc Contrib)]
    print("Word id, Topics      : ", c[1][:3])  # [(Word id, [Topics])]
    print("Phi Values (word id) : ", c[2][:2])  # [(Word id, [(Topic, Phi Value)])]
    print("Word, Topics         : ", [(dct[wd], topic) for wd, topic in c[1][:2]])   # [(Word, [Topics])]
    print("Phi Values (word)    : ", [(dct[wd], topic) for wd, topic in c[2][:2]])  # [(Word, [(Topic, Phi Value)])]
    print("------------------------------------------------------\n")

#> Document Topics      :  [(2, 0.96124125), (6, 0.038569752)]
#> Word id, Topics      :  [(0, [2, 6]), (7, [2, 6]), (10, [2, 6])]
#> Phi Values (word id) :  [(0, [(2, 2.887749), (6, 0.112249866)]), (7, [(2, 0.90105206), (6, 0.09893738)])]
#> Word, Topics         :  [('ability', [2, 6]), ('absurdity', [2, 6])]
#> Phi Values (word)    :  [('ability', [(2, 2.887749), (6, 0.112249866)]), ('absurdity', [(2, 0.90105206), (6, 0.09893738)])]
#> ------------------------------------------------------

#> Document Topics      :  [(6, 0.9997751)]
#> Word id, Topics      :  [(0, [6]), (10, [6]), (16, [6])]
#> Phi Values (word id) :  [(0, [(6, 5.9999967)]), (10, [(6, 2.9999983)])]
#> Word, Topics         :  [('ability', [6]), ('academic', [6])]
#> Phi Values (word)    :  [('ability', [(6, 5.9999967)]), ('academic', [(6, 2.9999983)])]
#> ------------------------------------------------------

#> Document Topics      :  [(6, 0.9998023)]
#> Word id, Topics      :  [(1, [6]), (10, [6]), (15, [6])]
#> Phi Values (word id) :  [(1, [(6, 0.99999917)]), (10, [(6, 5.999997)])]
#> Word, Topics         :  [('able', [6]), ('academic', [6])]
#> Phi Values (word)    :  [('able', [(6, 0.99999917)]), ('academic', [(6, 5.999997)])]
#> ------------------------------------------------------

('Document Topics      : ', [(2, 0.96452165), (6, 0.035288557)])
('Word id, Topics      : ', [(0, [2, 6]), (7, [2, 6]), (10, [2, 6])])
('Phi Values (word id) : ', [(0, [(2, 2.8961294), (6, 0.10216269)]), (7, [(2, 0.86925656), (6, 0.08887509)])])
('Word, Topics         : ', [(u'ability', [2, 6]), (u'absurdity', [2, 6])])
('Phi Values (word)    : ', [(u'ability', [(2, 2.8961294), (6, 0.10216269)]), (u'absurdity', [(2, 0.86925656), (6, 0.08887509)])])
------------------------------------------------------

('Document Topics      : ', [(6, 0.99933493)])
('Word id, Topics      : ', [(0, [6]), (10, [6]), (16, [6])])
('Phi Values (word id) : ', [(0, [(6, 5.994235)]), (10, [(6, 2.9965134)])])
('Word, Topics         : ', [(u'ability', [6]), (u'academic', [6])])
('Phi Values (word)    : ', [(u'ability', [(6, 5.994235)]), (u'academic', [(6, 2.9965134)])])
------------------------------------------------------

('Document Topics      : ', [(6, 0.9998019)])
('Word id, Topics      : ', [(1, [6]), (1

**Create a LSI topic model**

In [14]:
from gensim.models import LsiModel

# Build the LSI Model
lsi_model = LsiModel(corpus=corpus, id2word=dct, num_topics=7, decay=0.5)

# View Topics
print(lsi_model.print_topics(-1))
#> [(0, '0.262*"also" + 0.197*"state" + 0.197*"american" + 0.178*"first" + '
#>   '0.151*"many" + 0.149*"time" + 0.147*"year" + 0.130*"person" + 0.130*"world" '
#>   '+ 0.124*"war"'),
#>  (1, '0.937*"agave" + 0.164*"asia" + 0.100*"aruba" + 0.063*"plant" + 0.053*"var" '
#>   '+ 0.052*"state" + 0.045*"east" + 0.044*"congress" + -0.042*"first" + '
#>   '0.041*"maguey"'),
#>  (2, '0.507*"american" + 0.180*"football" + 0.179*"player" + 0.168*"war" + '
#>   '0.150*"british" + -0.140*"also" + 0.114*"ball" + 0.110*"day" + '
#>   '-0.107*"atheism" + -0.106*"god"'),
#>  (3, '-0.362*"apollo" + 0.248*"lincoln" + 0.211*"state" + -0.172*"player" + '
#>   '-0.151*"football" + 0.127*"union" + -0.125*"ball" + 0.124*"government" + '
#>   '-0.116*"moon" + 0.116*"jews"'),
#>  (4, '-0.363*"atheism" + -0.334*"god" + -0.329*"lincoln" + -0.230*"apollo" + '
#>   '-0.215*"atheist" + -0.143*"abraham" + 0.136*"island" + -0.132*"aristotle" + '
#>   '0.124*"aluminium" + -0.119*"belief"'),
#>  (5, '-0.360*"apollo" + 0.344*"atheism" + -0.326*"lincoln" + 0.226*"god" + '
#>   '0.205*"atheist" + 0.139*"american" + -0.130*"lunar" + 0.128*"football" + '
#>   '-0.125*"moon" + 0.114*"belief"'),
#>  (6, '-0.313*"lincoln" + 0.226*"apollo" + -0.166*"football" + -0.163*"war" + '
#>   '0.162*"god" + 0.153*"australia" + -0.148*"play" + -0.146*"ball" + '
#>   '0.122*"atheism" + -0.122*"line"')]

2019-08-27 18:39:49,585 : INFO : using serial LSI version on this node
2019-08-27 18:39:49,588 : INFO : updating model with new documents
2019-08-27 18:39:49,591 : INFO : preparing a new chunk of documents
2019-08-27 18:39:49,690 : INFO : using 100 extra samples and 2 power iterations
2019-08-27 18:39:49,693 : INFO : 1st phase: constructing (40074, 107) action matrix
2019-08-27 18:39:49,739 : INFO : orthonormalizing (40074, 107) action matrix
2019-08-27 18:39:50,750 : INFO : 2nd phase: running dense svd on (107, 100) matrix
2019-08-27 18:39:50,797 : INFO : computing the final decomposition
2019-08-27 18:39:50,798 : INFO : keeping 7 factors (discarding 62.871% of energy spectrum)
2019-08-27 18:39:50,844 : INFO : processed documents up to #100
2019-08-27 18:39:50,850 : INFO : topic #0(973.794): 0.262*"also" + 0.197*"state" + 0.197*"american" + 0.178*"first" + 0.151*"many" + 0.149*"time" + 0.147*"year" + 0.130*"person" + 0.130*"world" + 0.124*"war"
2019-08-27 18:39:50,852 : INFO : topic #

[(0, u'0.262*"also" + 0.197*"state" + 0.197*"american" + 0.178*"first" + 0.151*"many" + 0.149*"time" + 0.147*"year" + 0.130*"person" + 0.130*"world" + 0.124*"war"'), (1, u'0.937*"agave" + 0.164*"asia" + 0.100*"aruba" + 0.063*"plant" + 0.053*"var" + 0.052*"state" + 0.045*"east" + 0.044*"congress" + -0.042*"first" + 0.041*"maguey"'), (2, u'0.507*"american" + 0.180*"football" + 0.179*"player" + 0.168*"war" + 0.150*"british" + -0.140*"also" + 0.114*"ball" + 0.110*"day" + -0.107*"atheism" + -0.106*"god"'), (3, u'-0.362*"apollo" + 0.248*"lincoln" + 0.211*"state" + -0.172*"player" + -0.151*"football" + 0.127*"union" + -0.125*"ball" + 0.124*"government" + -0.116*"moon" + 0.116*"jews"'), (4, u'-0.363*"atheism" + -0.334*"god" + -0.329*"lincoln" + -0.230*"apollo" + -0.215*"atheist" + -0.143*"abraham" + 0.136*"island" + -0.132*"aristotle" + 0.124*"aluminium" + -0.119*"belief"'), (5, u'-0.360*"apollo" + 0.344*"atheism" + -0.326*"lincoln" + 0.226*"god" + 0.205*"atheist" + 0.139*"american" + -0.130*"